In [6]:
import pandas as pd
from functions import *

# Define global parameter ranges
PARAMETER_RANGES = {
    "Gaussian": {"param_name": "sigma", "values": [2.5, 3, 3.5]},
    "Median": {"param_name": "size", "values": [5, 8, 10]},
    "TV-Chambolle": {"param_name": "weight", "values": [0.05, 0.1, 0.2]},
    "Wavelet": {"param_name": None, "values": None},  # Use default parameters
    "NL-Means": {"param_name": None, "values": None}  # Use default parameters
}

disable_progress = True  # Set to False to enable progress bar

def main():
    """Main script to evaluate denoisers and save results."""
    # Get paths
    data_path, output_path = get_paths()

    # Choose denoiser
    denoiser_name = "NL-Means"  # "Gaussian", "Median", "TV-Chambolle", "Wavelet", or "NL-Means"

    # Process images with the chosen denoiser
    results_df, result_filename = process_with_denoiser(denoiser_name, data_path, num_images=2, parameter_ranges=PARAMETER_RANGES, disable_progress=disable_progress)

    # Save and display results
    display_styled_results(results_df, output_path, result_filename, title=f"{denoiser_name} Denoiser Results")

    # Compute and save averages
    avg_results = compute_averages(results_df)
    display_styled_results(avg_results, output_path, f"average_{result_filename}", title=f"Average {denoiser_name} Results")


if __name__ == "__main__":
    main()



NL-Means Denoiser Results:


,DenoiserType,Parameter,ImageIndex,Channel,PSNR,SI-PSNR,SSIM,Runtime,RAM Usage
0,NL-Means,Default parameters,001,0,-0.76,26.06,0.0001,0.5907 s,8.20 MB
1,NL-Means,Default parameters,001,1,-0.51,29.40,0.0001,0.5854 s,13.50 MB
2,NL-Means,Default parameters,001,2,-2.68,33.19,0.0001,0.5814 s,1.14 MB
3,NL-Means,Default parameters,002,0,8.87,30.42,0.0012,0.5790 s,10.28 MB
4,NL-Means,Default parameters,002,1,10.95,34.93,0.0041,0.5821 s,0.05 MB
5,NL-Means,Default parameters,002,2,7.81,34.37,0.0011,0.5799 s,9.00 MB


Results saved to /Users/Matthieu/Documents/GitHub/ML4Science-CS433/output/NL-Means_denoiser_results_default.csv

Average NL-Means Results:


,DenoiserType,Parameter,Channel,PSNR,SI-PSNR,SSIM,Runtime,RAM Usage
0,NL-Means,Default parameters,0,4.06,28.24,0.0006,0.5849 s,9.24 MB
1,NL-Means,Default parameters,1,5.22,32.17,0.0021,0.5838 s,6.77 MB
2,NL-Means,Default parameters,2,2.56,33.78,0.0006,0.5806 s,5.07 MB


Results saved to /Users/Matthieu/Documents/GitHub/ML4Science-CS433/output/average_NL-Means_denoiser_results_default.csv
